# New Section

In [0]:
!pwd

/content


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls /content/drive/My\ Drive/shreya_bert_on_collab/

checkpoints_bert_de_base	   checkpoints_bert_lat_multilingual_2
checkpoints_bert_lat_multilingual  checkpoints_bert_lat_multilingual_3


In [0]:
!nvidia-smi

Mon Dec  9 22:58:05 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#!rm -rf bert-nmt-latin

In [0]:
!git clone https://github.com/shreyapandit/bert-nmt-latin.git

In [0]:
mv  bert-nmt-latin/* .

#TODO: Upload latin files to translation folder (folder name: /content/examples/translation/lat-en_data)

In [0]:
!mkdir /content/examples/translation/lat-en_data

In [0]:
##TODO: Make sure you upload the files now before proceeding next

In [0]:
!ls /content/examples/translation/lat-en_data

test.tags.lat-en.en   train.tags.lat-en.en
test.tags.lat-en.lat  train.tags.lat-en.lat


In [0]:
!ls /content/examples/translation/lat-en_data

test.tags.lat-en.en   train.tags.lat-en.en
test.tags.lat-en.lat  train.tags.lat-en.lat


In [0]:
!ls /content/examples/translation/iwslt14.tokenized.de-en/

code			test.en-de.de.idx	 train.en-de.en.bin
dict.de.txt		test.en-de.en.bin	 train.en-de.en.idx
dict.en.txt		test.en-de.en.idx	 valid.bert.de-en
makedataforbert.sh	tmp			 valid.bert.en
preprocess.py		train.bert.de-en	 valid.bert.en-de.en.bin
test.bert.de-en		train.bert.en		 valid.bert.en-de.en.idx
test.bert.en		train.bert.en-de.en.bin  valid.de
test.bert.en-de.en.bin	train.bert.en-de.en.idx  valid.en
test.bert.en-de.en.idx	train.de		 valid.en-de.de.bin
test.de			train.en		 valid.en-de.de.idx
test.en			train.en-de.de.bin	 valid.en-de.en.bin
test.en-de.de.bin	train.en-de.de.idx	 valid.en-de.en.idx


In [0]:
!cp  examples/translation/iwslt14.tokenized.de-en/makedataforbert.sh examples/translation/makedataforbert.sh 

In [0]:
!mv examples/translation/iwslt14.tokenized.de-en examples/translation/iwslt14.tokenized.de-en_backup

In [0]:
# !rm /content/examples/translation/iwslt14.tokenized.de-en/tmp/*
# !rm /content/examples/translation/iwslt14.tokenized.de-en/*

In [0]:
#!ls examples/translation/iwslt14.tokenized.de-en/tmp

In [0]:
#!mv examples/translation/iwslt14.tokenized.de-en examples/translation/iwslt14.tokenized.de-en_backup

In [0]:
#!cat examples/translation/iwslt14.tokenized.de-en/tmp/train.tags.de-en.clean.en

In [0]:
#!cat examples/translation/iwslt14.tokenized.de-en/tmp/train.tags.de-en.clean.de

In [0]:
%%writefile examples/translation/prepare-wmt14_shreya.sh

#!/usr/bin/env bash
#
# Adapted from https://github.com/facebookresearch/MIXER/blob/master/prepareData.sh

echo 'Cloning Moses github repository (for tokenization scripts)...'
git clone https://github.com/moses-smt/mosesdecoder.git

echo 'Cloning Subword NMT repository (for BPE pre-processing)...'
git clone https://github.com/rsennrich/subword-nmt.git

SCRIPTS=mosesdecoder/scripts
TOKENIZER=$SCRIPTS/tokenizer/tokenizer.perl
LC=$SCRIPTS/tokenizer/lowercase.perl
CLEAN=$SCRIPTS/training/clean-corpus-n.perl
BPEROOT=subword-nmt
BPE_TOKENS=10000

if [ ! -d "$SCRIPTS" ]; then
    echo "Please set SCRIPTS variable correctly to point to Moses scripts."
    exit
fi

src=lat
tgt=en
lang=lat-en
prep=iwslt14.tokenized.lat-en
tmp=$prep/tmp
orig=lat-en_data

#mkdir -p $orig $tmp $prep
mkdir -p $tmp $prep
echo $orig $tmp $prep


#cd ..

echo "pre-processing train data..."
for l in $src $tgt; do
    f=train.tags.$lang.$l
    tok=train.tags.$lang.tok.$l

    cat $orig/$f | \
    perl $TOKENIZER -threads 8 -l $l > $tmp/$tok
    echo ""


done

perl $CLEAN -ratio 1.5 $tmp/train.tags.$lang.tok $src $tgt $tmp/train.tags.$lang.clean 1 175
for l in $src $tgt; do
    perl $LC < $tmp/train.tags.$lang.clean.$l > $tmp/train.tags.$lang.$l
done


# the code above does good job with creating tokens and clean token , 6 files are created..
# following code does something with test and validation set..


echo "pre-processing valid/test data..."
# for l in $src $tgt; do
#     for o in `ls $orig/$lang/IWSLT14.TED*.$l.xml`; do
#     fname=${o##*/}
#     f=$tmp/${fname%.*}
#     echo $o $f
#     grep '<seg id' $o | \
#         sed -e 's/<seg id="[0-9]*">\s*//g' | \
#         sed -e 's/\s*<\/seg>\s*//g' | \
#         sed -e "s/\’/\'/g" | \
#     perl $TOKENIZER -threads 8 -l $l | \
#     perl $LC > $f
#     echo ""
#     done
# done

for l in $src $tgt; do
    f=test.tags.$lang.$l
    tok=test.tags.$lang.tok.$l

    cat $orig/$f | \
    perl $TOKENIZER -threads 8 -l $l > $tmp/$tok
    echo ""

done

echo "creating train, valid, test..."
for l in $src $tgt; do
    awk '{if (NR%23 == 0)  print $0; }' $tmp/train.tags.$lang.$l > $tmp/valid.$l
    awk '{if (NR%23 != 0)  print $0; }' $tmp/train.tags.$lang.$l > $tmp/train.$l

    # cat $tmp/IWSLT14.TED.dev2010.de-en.$l \
    #     $tmp/IWSLT14.TEDX.dev2012.de-en.$l \
    #     $tmp/IWSLT14.TED.tst2010.de-en.$l \
    #     $tmp/IWSLT14.TED.tst2011.de-en.$l \
    #     $tmp/IWSLT14.TED.tst2012.de-en.$l \
    #     > $tmp/test.$l
    cat $orig/test.tags.$lang.$l  > $tmp/test.$l

done

TRAIN=$tmp/train.en-de
BPE_CODE=$prep/code
rm -f $TRAIN
for l in $src $tgt; do
    cat $tmp/train.$l >> $TRAIN
done

echo "learn_bpe.py on ${TRAIN}..."
python $BPEROOT/learn_bpe.py -s $BPE_TOKENS < $TRAIN > $BPE_CODE

for L in $src $tgt; do
    for f in train.$L valid.$L test.$L; do
        echo "apply_bpe.py to ${f}..."
        python $BPEROOT/apply_bpe.py -c $BPE_CODE < $tmp/$f > $prep/$f
    done
done

Writing examples/translation/prepare-wmt14_shreya.sh


In [0]:
!cd examples/translation/ && sh ./prepare-wmt14_shreya.sh

Cloning Moses github repository (for tokenization scripts)...
fatal: destination path 'mosesdecoder' already exists and is not an empty directory.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 538 (delta 10), reused 21 (delta 7), pack-reused 509
Receiving objects: 100% (538/538), 226.98 KiB | 605.00 KiB/s, done.
Resolving deltas: 100% (316/316), done.
lat-en_data iwslt14.tokenized.lat-en/tmp iwslt14.tokenized.lat-en
pre-processing train data...
Tokenizer Version 1.1
Language: lat
Number of threads: 8

Tokenizer Version 1.1
Language: en
Number of threads: 8

clean-corpus.perl: processing iwslt14.tokenized.lat-en/tmp/train.tags.lat-en.tok.lat & .en to iwslt14.tokenized.lat-en/tmp/train.tags.lat-en.clean, cutoff 1-175, ratio 1.5
......
Input sentences: 60134  Output sentences:  21581
pre-processing va

In [0]:
#!ls -lah  --time-style=full-iso examples/translation/iwslt14.tokenized.lat-en/

In [0]:
#!ls -lah  --time-style=full-iso examples/translation/iwslt14.tokenized.lat-en/tmp

In [0]:
#!ls -lah  --time-style=full-iso examples/translation/

In [0]:
!cp examples/translation/iwslt14.tokenized.de-en_backup/makedataforbert.sh \
      examples/translation/iwslt14.tokenized.lat-en/makedataforbert.sh

In [0]:
!cd examples/translation/iwslt14.tokenized.lat-en && sh ./makedataforbert.sh lat

src lng lat
Detokenizer Version $Revision: 4134 $
Language: lat
Detokenizer Version $Revision: 4134 $
Language: lat
Detokenizer Version $Revision: 4134 $
Language: lat


In [0]:
#!ls -lah  --time-style=full-iso examples/translation/iwslt14.tokenized.lat-en

In [0]:
!cd examples/translation/iwslt14.tokenized.lat-en && sh ./makedataforbert.sh en

src lng en
Detokenizer Version $Revision: 4134 $
Language: en
Detokenizer Version $Revision: 4134 $
Language: en
Detokenizer Version $Revision: 4134 $
Language: en


In [0]:
#!ls -lah  --time-style=full-iso examples/translation/iwslt14.tokenized.lat-en

In [0]:
!pip install --editable .

Obtaining file:///content
  Running setup.py develop for fairseq


In [0]:
#!rm  /content/examples/translation/iwslt14.tokenized.lat-en/*

In [0]:
!cp examples/translation/iwslt14.tokenized.de-en_backup/preprocess.py \
      examples/translation/iwslt14.tokenized.lat-en/preprocess.py


In [0]:
# Remove if running again..
!rm /content/examples/translation/iwslt14.tokenized.lat-en/dict.en.txt
!rm /content/examples/translation/iwslt14.tokenized.lat-en/dict.lat.txt

rm: cannot remove '/content/examples/translation/iwslt14.tokenized.lat-en/dict.en.txt': No such file or directory
rm: cannot remove '/content/examples/translation/iwslt14.tokenized.lat-en/dict.lat.txt': No such file or directory


In [0]:
!cd examples/translation/iwslt14.tokenized.lat-en/ &&   python preprocess.py --source-lang en --target-lang lat \
  --trainpref train --validpref valid --testpref test \
  --destdir `pwd`  --joined-dictionary --bert-model-name bert-base-multilingual-uncased

Namespace(alignfile=None, bert_model_name='bert-base-multilingual-uncased', cpu=False, criterion='cross_entropy', dataset_impl='cached', destdir='/content/examples/translation/iwslt14.tokenized.lat-en', fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=True, log_format=None, log_interval=1000, lr_scheduler='fixed', memory_efficient_fp16=False, min_loss_scale=0.0001, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=False, optimizer='nag', padding_factor=8, seed=1, source_lang='en', srcdict=None, target_lang='lat', task='translation', tbmf_wrapper=False, tensorboard_logdir='', testpref='test', tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, trainpref='train', user_dir=None, validpref='valid', workers=1)
{'train.en', 'train.lat'}
| [en] Dictionary: 9727 types
| [en] train.en: 20643 sents, 1342813 tokens, 0.0% replaced by <unk>
| [en] Dictionary: 9727 types
| [en] valid.en: 938 sents, 62438 tokens, 0.

In [0]:
#!ls /content/examples/translation/iwslt14.tokenized.lat-en/

In [0]:
#!ls -lah  --time-style=full-iso examples/translation/iwslt14.tokenized.lat-en/

In [0]:
#!rm checkpoints

In [0]:
#!ln -s  checkpoints/ /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual

In [0]:
%%writefile training_script.sh
#!/usr/bin/env bash
nvidia-smi

cd . #/yourpath/bertnmt
python3 -c "import torch; print(torch.__version__)"

src=en
tgt=lat
bedropout=0.5
ARCH=transformer_s2_iwslt_de_en
DATAPATH=examples/translation/iwslt14.tokenized.$tgt-$src
#SAVEDIR=checkpoints/iwed_${src}_${tgt}_${bedropout}
#SAVEDIR=/content/drive/My*/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2
#mkdir -p $SAVEDIR
mkdir /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2 
if [ ! -f $SAVEDIR/checkpoint_nmt.pt ]
then
    cp /your_pretrained_nmt_model $SAVEDIR/checkpoint_nmt.pt
fi
if [ ! -f "$SAVEDIR/checkpoint_last.pt" ]
then
## incorrectly coming to this
echo "commented code for warmup-from-nmt "
warmup=""
#warmup="--warmup-from-nmt --reset-lr-scheduler"
else
warmup=""
fi


python train.py $DATAPATH \
-a $ARCH --optimizer adam --lr 0.001 -s $src -t $tgt --label-smoothing 0.1 \
--dropout 0.3  --min-lr '1e-09' --lr-scheduler inverse_sqrt --weight-decay 0.0001 \
--criterion label_smoothed_cross_entropy --max-update 150000 --warmup-updates 4000 --warmup-init-lr '1e-07' \
--adam-betas '(0.9,0.98)' \
--save-dir /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2 \
\--share-all-embeddings $warmup \
--encoder-bert-dropout --encoder-bert-dropout-ratio $bedropout \
--save-interval 5 --max-epoch 20  --bert-model-name bert-base-multilingual-uncased --max-tokens 4000 \
 | tee -a /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2/training.log

Overwriting training_script.sh


In [0]:

!sh ./training_script.sh

Mon Dec  9 15:30:41 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
#!ls -lah checkpoints/iwed_en_lat_0.5/

In [0]:
!python generate.py  --quiet --bert-model-name bert-base-multilingual-uncased \
                    --path /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2/checkpoint_best.pt \
                    examples/translation/iwslt14.tokenized.lat-en \
                    | tee -a /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2/generate.log

Namespace(beam=5, bert_first=True, bert_gates=[1, 1, 1, 1, 1, 1], bert_model_name='bert-base-multilingual-uncased', bert_output_layer=-1, bert_ratio=1.0, change_ratio=False, cpu=False, criterion='cross_entropy', data='examples/translation/iwslt14.tokenized.lat-en', dataset_impl='cached', decoder_no_bert=False, diverse_beam_groups=-1, diverse_beam_strength=0.5, encoder_bert_dropout=False, encoder_bert_dropout_ratio=0.25, encoder_bert_mixup=False, encoder_ratio=1.0, finetune_bert=False, force_anneal=None, fp16=False, fp16_init_scale=128, fp16_scale_tolerance=0.0, fp16_scale_window=None, gen_subset='test', lazy_load=False, left_pad_source='True', left_pad_target='False', lenpen=1, log_format=None, log_interval=1000, lr_scheduler='fixed', lr_shrink=0.1, mask_cls_sep=False, match_source_len=False, max_len_a=0, max_len_b=200, max_sentences=None, max_source_positions=1024, max_target_positions=1024, max_tokens=12000, memory_efficient_fp16=False, min_len=1, min_loss_scale=0.0001, model_overrid

In [0]:
!ls 

In [0]:
%%writefile training_script.sh
#!/usr/bin/env bash
nvidia-smi

cd . #/yourpath/bertnmt
python3 -c "import torch; print(torch.__version__)"

src=en
tgt=lat
bedropout=0.5
ARCH=transformer_s2_iwslt_de_en
DATAPATH=examples/translation/iwslt14.tokenized.$tgt-$src
#SAVEDIR=checkpoints/iwed_${src}_${tgt}_${bedropout}
#SAVEDIR=/content/drive/My*/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2
#mkdir -p $SAVEDIR
mkdir /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2 
if [ ! -f $SAVEDIR/checkpoint_nmt.pt ]
then
    cp /your_pretrained_nmt_model $SAVEDIR/checkpoint_nmt.pt
fi
if [ ! -f "$SAVEDIR/checkpoint_last.pt" ]
then
## incorrectly coming to this
echo "commented code for warmup-from-nmt "
warmup=""
#warmup="--warmup-from-nmt --reset-lr-scheduler"
else
warmup=""
fi

python train.py $DATAPATH \
-a $ARCH --optimizer adam --lr 0.0005 -s $src -t $tgt --label-smoothing 0.1 \
--dropout 0.3  --min-lr '1e-09' --lr-scheduler inverse_sqrt --weight-decay 0.0001 \
--criterion label_smoothed_cross_entropy --max-update 150000 --warmup-updates 4000 --warmup-init-lr '1e-07' \
--adam-betas '(0.9,0.98)' \
--save-dir /content/drive/My\ Drive/shreya_bert_on_collab/checkpoints_bert_lat_multilingual_2 \
\--share-all-embeddings $warmup \
--encoder-bert-dropout --encoder-bert-dropout-ratio $bedropout \
--save-interval 5 --max-epoch 40  --bert-model-name bert-base-multilingual-uncased --max-tokens 4000 | tee -a $SAVEDIR/training.log

In [0]:
!sh ./training_script.sh

In [0]:
!python generate.py  --quiet --bert-model-name bert-base-multilingual-uncased \
                    --path /content/checkpoints/iwed_en_lat_0.5/checkpoint_best.pt \
                    examples/translation/iwslt14.tokenized.lat-en

In [0]:
!wc -l examples/translation/iwslt14.tokenized.ita-en_backup/train.en

160239 examples/translation/iwslt14.tokenized.de-en_backup/train.en


In [0]:
!wc -l examples/translation/iwslt14.tokenized.ita-en_backup/test.en

6750 examples/translation/iwslt14.tokenized.de-en_backup/test.en


In [0]:
!while true; do date -uIs ; sleep 5m ; done

2019-12-08T21:58:32+00:00
2019-12-08T22:03:32+00:00
2019-12-08T22:08:32+00:00
2019-12-08T22:13:32+00:00
2019-12-08T22:18:32+00:00
2019-12-08T22:23:32+00:00
2019-12-08T22:28:32+00:00
2019-12-08T22:33:32+00:00
2019-12-08T22:38:32+00:00
2019-12-08T22:43:32+00:00
2019-12-08T22:48:32+00:00
2019-12-08T22:53:32+00:00
2019-12-08T22:58:32+00:00
2019-12-08T23:03:32+00:00
2019-12-08T23:08:32+00:00
2019-12-08T23:13:32+00:00
2019-12-08T23:18:32+00:00
2019-12-08T23:23:32+00:00
2019-12-08T23:28:32+00:00
2019-12-08T23:33:32+00:00
2019-12-08T23:38:32+00:00
2019-12-08T23:43:32+00:00


KeyboardInterrupt: ignored